In [8]:
import sqlalchemy as db
import pandas as pd

In [45]:
engine = db.create_engine('sqlite:///scraped_jobs.sqlite')

In [46]:
connection = engine.connect()


In [63]:
metadata = db.MetaData()

salaries = db.Table('salaries', metadata,
                   db.Column('Job_Id', db.Integer(), primary_key=True),
                   db.Column('Job_Name', db.String(255), nullable = False),
                   db.Column('Salary', db.Integer()))

salary_period = db.Table('salary_period', metadata,
                   db.Column('Job_Id', db.Integer(), foreign_key=True),
                   db.Column('Salary_Time_Period', db.String(255), nullable = True),
                   db.Column('Salary_Converted_Hourly', db.Integer()),
                   db.Column('Salary_Id', db.Integer(), primary_key=True))

job_description = db.Table('job_description', metadata,
                   db.Column('Job_Id', db.Integer(), foreign_key=True),
                   db.Column('Company_Name', db.String(255), nullable=False),
                   db.Column('Job_Description', db.String()),
                   db.Column('Descr_Id', db.Integer(), primary_key=True))

job_search = db.Table('job_search', metadata,
                   db.Column('Job_Id', db.Integer(), foreign_key=True),
                   db.Column('searched__data_scientist', db.Boolean()),
                   db.Column('searched__machine_learning_engineer', db.Boolean()),
                   db.Column('searched_data_analyst', db.Boolean(), ))

metadata.create_all(engine)

/Users/ipreoteasa/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sqlalchemy/sql/base.py:302: SAWarning: Can't validate argument 'foreign_key'; can't locate any SQLAlchemy dialect named 'foreign'
  % (k, dialect_name)


In [86]:
pd.read_sql_query('SELECT Salary FROM salaries', connection)

,Salary
0,39853.0
1,NaN
2,NaN
3,NaN
4,35846.0
...,...
2206,60000.0
2207,NaN
2208,95000.0
2209,300.0


In [50]:
df = pd.read_csv('../clean_data/jobs.csv', index_col=0)
# df.rename({'Unnamed: 0': 'Job Id'},)
df.head(2)

,company,job_title,job_descr,job_url,job_location,searched_data_analyst,searched__data_scientist,searched__machine_learning_engineer,salary_from_page_source_as_stated,salary_from_page_source_conv_hourly,salary_from_page_source_time_period,Num_reviews,id
0,Medicines and Healthcare products Regulatory A...,CPRD Data Scientist,would like reassure all applicant that recruit...,https://www.indeed.co.uk/pagead/clk?mo=r&ad=-6...,NaN,0,1,0,39853.0,21.0,Y,18.0,0
1,PwC,Deals - Investigative Analytics - Data Scienti...,career within Forensics Technology service wil...,https://www.indeed.co.uk/rc/clk?jk=7548e60a487...,NaN,0,1,0,NaN,NaN,NaN,7678.0,1


In [51]:
salaries_df = df[['id', 'job_title', 'salary_from_page_source_as_stated']]
salaries_df.head()

,id,job_title,salary_from_page_source_as_stated
0,0,CPRD Data Scientist,39853.0
1,1,Deals - Investigative Analytics - Data Scienti...,NaN
2,2,Data Scientist - Cyber,NaN
3,3,Lead Data Scientist,NaN
4,4,G4 Lead Data Scientist - Cyber,35846.0


In [52]:
salaries_df.rename({'id': 'Job_Id', 'job_title': 'Job_Name', 'salary_from_page_source_as_stated':'Salary'}, axis=1, inplace=True)

/Users/ipreoteasa/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [53]:
salaries_df.head()

,Job_Id,Job_Name,Salary
0,0,CPRD Data Scientist,39853.0
1,1,Deals - Investigative Analytics - Data Scienti...,NaN
2,2,Data Scientist - Cyber,NaN
3,3,Lead Data Scientist,NaN
4,4,G4 Lead Data Scientist - Cyber,35846.0


In [54]:
salaries_df.to_sql('salaries', 
                  connection,
                  if_exists='append',index=False)

In [84]:
pd.read_sql_query('''SELECT Job_Name, Salary 
                  FROM salaries 
                  WHERE Salary IS NOT NULL & Salary > 40000 & Job_Name REGEXP 'D'
                  LIMIT 10''', 
                  connection)

,Job_Name,Salary
